In [11]:
import pandas as pd

df = pd.read_csv("online_retail_limpio.csv", parse_dates=["InvoiceDate"])

**CREAMOS COLUMNAS NUEVAS PARA LAS ALERTAS**

In [12]:
# Filtro las cancelaciones explícitas
df["IsCancellation"] = df["InvoiceNo"].astype(str).str.startswith("C")

# Agrego la columna de semana
df["Week"] = df["InvoiceDate"].dt.to_period("W").astype(str)

# Calculo las métricas por producto y semana
weekly_data = df.groupby(["StockCode", "Week"]).agg({
    "Quantity": "sum",
    "UnitPrice": "mean",
    "IsCancellation": "mean",
    "Country": lambda x: x.mode()[0] if not x.mode().empty else None
}).reset_index()

# Calculo el valor total de lo vendido
weekly_data["TotalValue"] = weekly_data["Quantity"] * weekly_data["UnitPrice"]

**DEFINIMOS LAS REGLAS DE LAS ALERTAS**

In [13]:
# Regla 1: Demanda semanal > 200 unidades para generar alerta de demanda elevada.
rule_1 = weekly_data[weekly_data["Quantity"] > 200].copy()
rule_1["Alerta"] = "Demanda semanal alta"

In [14]:
# Regla 2: Caída de demanda > 70% para generar una alerta de caída de demanda.
monthly_avg = df.copy()
monthly_avg["Month"] = df["InvoiceDate"].dt.to_period("M").astype(str)
monthly_avg = monthly_avg.groupby(["StockCode", "Month"]).agg({"Quantity": "sum"}).groupby("StockCode").mean().reset_index()
monthly_avg.columns = ["StockCode", "MonthlyAvgQty"]

rule_2 = weekly_data.merge(monthly_avg, on="StockCode", how="left")
rule_2 = rule_2[rule_2["Quantity"] < 0.3 * rule_2["MonthlyAvgQty"]].copy()
rule_2["Alerta"] = "Caida de demanda significativa"

In [15]:
# Regla 3: Cancelaciones frecuentes > 30%. Alerta de problemas posibles problemas de calidad o logística con los productos.
rule_3 = weekly_data[weekly_data["IsCancellation"] > 0.3].copy()
rule_3["Alerta"] = "Alto indice de cancelaciones"

In [16]:
# Regla 4: Valor vendido > £1000. Alerta de producto de alto valor en riesgo de agotarse que requieren control más estricto.
rule_4 = weekly_data[weekly_data["TotalValue"] > 1000].copy()
rule_4["Alerta"] = "Valor economico alto en stock vendido"

In [17]:
# Combino todas las alertas (sin duplicados)
alerts = pd.concat([rule_1, rule_2, rule_3, rule_4], ignore_index=True)

In [18]:
alerts 

,StockCode,Week,Quantity,UnitPrice,IsCancellation,Country,TotalValue,Alerta,MonthlyAvgQty
0,10133,2011-08-15/2011-08-21,245,0.420000,0.0,United Kingdom,102.900000,Demanda semanal alta,NaN
1,10133,2011-08-22/2011-08-28,225,0.420000,0.0,United Kingdom,94.500000,Demanda semanal alta,NaN
2,10135,2010-12-13/2010-12-19,300,0.420000,0.0,United Kingdom,126.000000,Demanda semanal alta,NaN
3,10135,2011-02-21/2011-02-27,210,1.155000,0.0,Germany,242.550000,Demanda semanal alta,NaN
4,11001,2011-08-01/2011-08-07,384,1.270000,0.0,United Kingdom,487.680000,Demanda semanal alta,NaN
...,...,...,...,...,...,...,...,...,...
61750,POST,2011-11-07/2011-11-13,84,20.142857,0.0,France,1692.000000,Valor economico alto en stock vendido,NaN
61751,POST,2011-11-14/2011-11-20,154,22.509434,0.0,Germany,3466.452830,Valor economico alto en stock vendido,NaN
61752,POST,2011-11-21/2011-11-27,94,29.966129,0.0,Germany,2816.816129,Valor economico alto en stock vendido,NaN
61753,POST,2011-11-28/2011-12-04,109,21.575000,0.0,France,2351.675000,Valor economico alto en stock vendido,NaN


In [19]:
alerts.to_csv("online_retail_alertas.csv", index=False)